In [1]:
%reload_ext autoreload
%autoreload 2
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io.wavfile as wv
import scipy.signal as sig
import wave

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense

from util import util
from util import WavFileParts
from util.logUtil import LOG, LOG_HEADER
from util.confusionMatrix import ConfusionMatrix


Using TensorFlow backend.


### globale settings

In [2]:
classes = ['music', 'voice', 'environment']

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir

baseTargetDir = '/Users/etto/Desktop/pData'
baseTargetDir = 'E:/SAA_DATA/targetDir'

baseSrcDir = osDir + 'localizationFiles/20171025AllExtractionsMic4'
orgWavDirs1 = ['G428_0.0_1.4',
              'G527_0.5_1.4',
              'Studio_2.0_4.2'
              ]

orgWavDirs2 = ['G428_2.1_2.4',
              'G527_1.2_5.8',
              'Studio_3.0_2.0'
              ]

NFFT = 1024

storageFolder = '../storedData/'
chunksBaseDir = 'chunks'
rooms = ['Studio', 'G428', 'G527']

### utility functies

In [3]:
def readSoundChunksDynamic(moduleString):
    chunks = importlib.import_module(moduleString).soundChunks
    wfPts = []
    for jsonString in chunks:
        wfPts.append(WavFileParts.WavFilePartFromJson(jsonString))
    return wfPts

def timeFunction(func):
    """
    Aanroep: bijv. fpc = timeFunction(lambda: getFilesPerCategory(srcDir))
    """
    startTime = datetime.now()
    print('Start: ' + startTime.strftime('%H:%M:%S') + '\n=================')

    res = func()
    
    endTime = datetime.now()
    print('\n=================\nEnd: ' + endTime.strftime('%H:%M:%S'))
    print('Time taken: '),
    print(endTime - startTime)
    print()
    
    return res
    
def storeTestData(allSpectros, allClasses, storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    df = pd.DataFrame(allSpectros)
    df.to_hdf(path_or_buf=filename, key='spectros_' + keyName)

    df = pd.DataFrame(allClasses)
    df.to_hdf(path_or_buf=filename, key='classes_' + keyName)

def retrieveTestData(storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    specDf = pd.read_hdf(path_or_buf=filename, key='spectros_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return specDf, classesDf
    

### functies tbv trainen

In [4]:
# Maakt een dictionary aan; per categorie alle files (volledig pad) uit de srcDir
# srcDir is een van de orgWavDirs, bijvoorbeeld
#    localizationFiles/20171025AllExtractionsMic4/G428_0.0_1.4
def getFilesPerCategory(srcDir):
    filesPerCategory = {}
    for catDirLong in glob.glob(srcDir + '/*'):
        catDir = catDirLong.replace('\\', '/')
        catDir = catDir.replace(srcDir + '/', '')

        filesPerCategory[catDir] = []
        for filename in glob.glob(catDirLong + '/*'):
            filename = filename.replace('\\','/')
            filesPerCategory[catDir].append(filename)
    return filesPerCategory

def getFilesPerCatFromMultipleDirs(srcDirs, srcDirsBase=''):
    filesPerCat = {}
    for dirName in srcDirs:
        srcDir = srcDirsBase + '/' + dirName
        fpcNw = getFilesPerCategory(srcDir)
        if not filesPerCat:
            filesPerCat = fpcNw
        else:
            for key in filesPerCat:
                filesPerCat[key] += fpcNw[key]
    return filesPerCat
        

In [5]:
# Maakt een dictionary aan; per categorie de spectrogrammen
def getSpectrosFromFilesPerCategory(filesPerCategory):
    spectros = {}
    for clz in classes:
        spectros[clz] = []
        for filename in filesPerCategory[clz]:
            fs, signal = wv.read(filename)
            freq_array, segment_times, spectrogram = sig.spectrogram(x=signal, fs=fs, nfft=NFFT, noverlap=0)
            spectros[clz].append(spectrogram.T)
    return spectros

In [6]:
def getClassLengths(spectrosPerCat):
    clzLengths = {}
    for clz in classes:
        clzLengths[clz] = sum([np.shape(lst)[0] for lst in spectrosPerCat[clz]])
    return clzLengths

In [7]:
# verwacht invoer van getSpectrosFromFilesPerCategory
# levert traindata op (X_train en Y_train)
def createTrainDataFromSpectros(spectrosPerCat, clzLengths):
    X_train = np.concatenate(spectrosPerCat[classes[0]], axis=0)
    for i in range(1, len(classes)):
        nwSpectros = np.concatenate(spectrosPerCat[classes[i]], axis=0)
        X_train = np.concatenate((X_train,nwSpectros), axis=0)
    
    # one-hot encoding voor Y_train
    nrFiles = clzLengths[classes[0]]
    Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

    nrFiles = clzLengths[classes[1]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.ones(nrFiles), np.zeros(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

    nrFiles = clzLengths[classes[2]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.zeros(nrFiles), np.ones(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
    
    return X_train, Y_train

### deep learning model

In [8]:
def create_model(layersizes):
    # create model
    model = Sequential()
    model.add(Dense(layersizes[0], input_dim=513, activation='relu'))
    for lsize in layersizes[1:]:
        model.add(Dense(lsize, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
def getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs):
    modelFilename = modelFilePath + baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    modelFilename += '.hd5'
    return modelFilename

In [10]:
def train_model(X_train, Y_train, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    history = timeFunction(lambda: soundModel.fit(X_train,Y_train, epochs=nrEpochs, shuffle=True, verbose=1))
    soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))
    plt.plot(history.history['loss'])
    plt.show()

In [11]:
def evaluate_model(X_test, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = load_model(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))

    # predicted classes
    predictions = soundModel.predict(X_test)
    predClasses = predictions.argmax(axis=1)

    matrix = ConfusionMatrix(classes)
    for vals in zip(realClasses, predClasses):
        matrix.add(int(vals[0]), int(vals[1]), 1)
    LOG(matrix.toString(),True)
    LOG('', True)
    LOG(matrix.toF1String(), True)
    return matrix

In [12]:
def train_and_evaluate_per_epoch(X_train, Y_train, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    for epNr in range(1, nrEpochs+1):
        LOG('\n*****************\n* Epoch nr {}\n*****************\n'.format(epNr), True)
        soundModel.fit(X_train,Y_train, epochs=1, shuffle=True, verbose=1)
        soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, epNr))
        evaluate_model(X_train, realClasses, layers, epNr, modelFilePath, baseModelFilename)

### trainen van model

In [13]:
# bepaal train data
srcDir = baseSrcDir + '/' + orgWavDirs1[0]
# fpc = getFilesPerCategory(srcDir)
fpc = getFilesPerCatFromMultipleDirs(orgWavDirs1 + orgWavDirs2, baseSrcDir)
spcs = getSpectrosFromFilesPerCategory(fpc)
clzLengths = getClassLengths(spcs)
X_train, Y_train = createTrainDataFromSpectros(spcs, clzLengths)
# real train classes
realTrainClasses = np.concatenate((np.zeros(clzLengths[classes[0]]), 
                              np.ones(clzLengths[classes[1]]), 
                              2*np.ones(clzLengths[classes[2]])))


In [14]:
baseModelFilename = '20180720_allOrgs'
modelFilePath = storageFolder
layers = [100, 20]

LOG_HEADER('Alle orgs, lagen: [100, 20]', True)

nrEpochs = 5
train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Alle orgs, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
1108305/1108305 [==============================] - 113s 102us/step - loss: 0.1799 - acc: 0.9338
                   m       v       e |  sens   acc
music         351223   14553     760 |  0.96  0.97
voice          18017  339738    8968 |  0.93  0.96
environment     1291    8015  365740 |  0.98  0.98
--------------------------------------
prec            0.95    0.94    0.97

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.93
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
1108305/1108305 [==============================] - 111s 100us/step - loss: 0.1163 - acc: 0.9569
                   m       v       e |  sens   acc
music         350210   15815     511 |  0.96  0.98
voice          10001  348236    8486 |  0.95  0.96
environment      898    6086  368062 |  0.98  0.99
-----------------------

In [15]:
baseModelFilename = '20180720_allOrgs'
modelFilePath = storageFolder
layers = [400, 250, 100, 20]

LOG_HEADER('Alle orgs, lagen: [400, 250, 100, 20]', True)

nrEpochs = 5
train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Alle orgs, lagen: [400, 250, 100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
1108305/1108305 [==============================] - 171s 154us/step - loss: 0.1454 - acc: 0.9462
                   m       v       e |  sens   acc
music         349038   16865     633 |  0.95  0.98
voice           8966  351792    5965 |  0.96  0.96
environment      852    9744  364450 |  0.97  0.98
--------------------------------------
prec            0.97    0.93    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98


*****************
* Epoch nr 2
*****************

Epoch 1/1
1108305/1108305 [==============================] - 174s 157us/step - loss: 0.1049 - acc: 0.9611
                   m       v       e |  sens   acc
music         349291   16613     632 |  0.95  0.98
voice           7034  352748    6941 |  0.96  0.97
environment      654    7602  366790 |  0.98  0.99
-------------

In [16]:
baseModelFilename = '20180720_allOrgs'
modelFilePath = storageFolder
layers = [400, 300, 200, 100, 50, 20, 10]

LOG_HEADER('Alle orgs, lagen: [400, 300, 200, 100, 50, 20, 10]', True)

nrEpochs = 5
train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Alle orgs, lagen: [400, 300, 200, 100, 50, 20, 10]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
1108305/1108305 [==============================] - 256s 231us/step - loss: 0.1500 - acc: 0.9447
                   m       v       e |  sens   acc
music         351060   14902     574 |  0.96  0.97
voice          12137  345444    9142 |  0.94  0.96
environment     1391    7256  366399 |  0.98  0.98
--------------------------------------
prec            0.96    0.94    0.97

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98


*****************
* Epoch nr 2
*****************

Epoch 1/1
1108305/1108305 [==============================] - 256s 231us/step - loss: 0.1077 - acc: 0.9603
                   m       v       e |  sens   acc
music         354786   11054     696 |  0.97  0.98
voice          13582  348212    4929 |  0.95  0.96
environment      564    9527  364955 |  0.97  0.99


In [17]:
baseModelFilename = '20180720_allOrgs'
modelFilePath = storageFolder
layers = [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]

LOG_HEADER('Alle orgs, lagen: [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]', True)

nrEpochs = 5
train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Alle orgs, lagen: [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
1108305/1108305 [==============================] - 353s 319us/step - loss: 0.1617 - acc: 0.9418
                   m       v       e |  sens   acc
music         348304   17826     406 |  0.95  0.97
voice          10841  342199   13683 |  0.93  0.96
environment     2216    4614  368216 |  0.98  0.98
--------------------------------------
prec            0.96    0.94    0.96

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
1108305/1108305 [==============================] - 351s 317us/step - loss: 0.1228 - acc: 0.9569
                   m       v       e |  sens   acc
music         355192   10856     488 |  0.97  0.98
voice          14876  342950    8897 |  0.94  0.96
environment     1090    5267  36868

### functies tbv testen